TDSE Project

$\;$

importing libaries

In [2]:
import math
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc

creating a gaussian shaped initial wavefunction

In [13]:
def initial_wavefunction(a,x):
    return math.exp(-a * (x ** 2))
wav = [initial_wavefunction(0.01,i) for i in range(-10,10)]

converting a wavefunction into a probability density function

In [15]:
def probability_density(wavefunction):
    wavefunction = np.array(wavefunction)
    conjugate_wavefunction = np.conjugate(wavefunction)
    probability_density = wavefunction * conjugate_wavefunction
    probability_density_real = list(probability_density.real)
    return probability_density_real
prob = probability_density(wav)

applying normalisation condition to a probability density function

In [18]:
def normalisation(prob_density, x_gap):
    total = np.sum(np.array(prob_density) * x_gap)
    normalised_prob_density = []
    for point in prob_density:
        point /= total
        normalised_prob_density.append(point)
    return normalised_prob_density

defining the first order step in time for a point in space

In [19]:
def first_order(i, delta_x, delta_t, old_wavefunction):
    return complex(0,1) * delta_t * ((old_wavefunction[i-1] - (2 * old_wavefunction[i]) + old_wavefunction[i + 1])/(2 * (delta_x**2)))

defining a second order step in time for a point in space

In [20]:
def second_order(i, delta_x, delta_t, old_wavefunction):
    return complex(0,1) * delta_t * ((first_order(i-1, delta_x, delta_t, old_wavefunction) - (2 * first_order(i, delta_x, delta_t, old_wavefunction)) + first_order(i+1, delta_x, delta_t, old_wavefunction))/(2 * (delta_x**2)))

applying a first order step in time to every point on a wavefunction

In [23]:
def time_step_forward_first(delta_x, delta_t, old_wavefunction):
    new_wavefunction = []
    old_wavefunction = [0] + old_wavefunction + [0]
    for i in range(1, (len(old_wavefunction) - 1)):
        new_wavefunction.append(old_wavefunction[i] + first_order(i, delta_x, delta_t, old_wavefunction))
    new_probability_density = normalisation(probability_density(new_wavefunction),1)
    return new_probability_density

applying a second order step in time to every point on a wavefunction

In [24]:
def time_step_forward_second(delta_x, delta_t, old_wavefunction):
    new_wavefunction = []
    old_wavefunction = [0] + [0] + old_wavefunction + [0] + [0]
    for i in range(2, (len(old_wavefunction) - 2)):
        new_wavefunction.append(old_wavefunction[i] + first_order(i, delta_x, delta_t, old_wavefunction) + (0.5 * second_order(i, delta_x, delta_t, old_wavefunction) * (delta_t**2)))
    new_probability_density = normalisation(probability_density(new_wavefunction),1)
    return new_probability_density

creating a set of axes for the animation

In [25]:
fig = plt.figure(figsize = (12,12))
ax = plt.axes(xlim=(-1, 1), ylim=(0, 0.0025))
plt.title('Evolution of the Probability Density for a Free Particle with Time')
plt.xlabel('Position')
plt.ylabel('Probability Density')
line, = ax.plot([], [], lw=2)
plt.close()

initialising the animation

In [28]:
def init():
    line.set_data([], [])
    return line,

defining a set of frames for the animation

In [26]:
positions = [0.001 * i for i in range(-5000,5001)]
initial_wavefunction = [initial_wavefunction(5,i) for i in positions]

def animate(i):
    x = positions
    y = time_step_forward_second(0.25, i, initial_wavefunction)
    line.set_data(x, y)
    return line,

outputting the animation

In [29]:
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=60, interval=150, blit=True)
%matplotlib inline
rc('animation', html='html5')
anim